#  Дрыц-тыц холодильник и шкафы

## Описание


### Как всё будет работать

Заходим в бот телеграмм
Отправляем голосове сообщение, в коором перечисляем, что есть в холодильнике и на кухне в шкафах
Сообщение распознаётся и выдаётся текст (и запрос на подтверждение правильности распознавания)
ДАлее выводится список подходящих рецептов с гиперссылками или же сообщение о том, что ничего не нашлось

### Технические моменты

**Сервер**
* Сначала попробовать работу MVP на Google Colab, потом на Raspberry Pi
* Gjnjv переложить, например, в Yandex Cloud, reg.ru облако и др.
* Как выкладывать на облако - см. курс на Stepik про телеграмм-бот для создания канала новостей

**Распознавание речи**
* отправлять голосове сообщение удобнее, чем стоаять и печатать
* см. проект `telegram_bot_voice_to_text`

**Взаимодействие с пользователем**
* Спросить правльно ли распознался список продуктов 
    - Вывести нумерованный список.
    - Если неправильно, то дать возможность пользователю откорректировать ингридиент, набрав номер в списке и введя новое название.
* Спросить про варианты подбора рецепта: 
    - полное совпадение;
    - недостаток 1 иингридиента;
    - недостаток 2 ингридиентов.

**Логотип**
* Логотип бота сгенерировать в MidJourney / Kandinsy / Шедеврум 

**Данные**
* набрать базу из say7 и т.п. - потом на них законно ссылаться без нарушеняи авторских прав
* можно даже ей предложить сотрудничество - она рекламирует бот, а внём только её рецепты

**Тесты**
* писать заранее - так приучать себя к test driven development (на этапе MVP в Google Colab - это будут простые assert statements или вообще простая проверка на равенство в ячейках)

**Потенциальные проблемы**
* подход с созданием `set` из слов - как быть с названиями ингридиентов из 2 и более слов типа `маринованные огурцы`, `лук репчатый`? Приводить всё к именительному падежу с помощью какой-то библиотеки? 
* Как искать искать игридиены-заменители типа `лук`, `лук репчатый`? Как не спутать `лук` с `перья лука`?

## Библиотеки

In [3]:
import requests
import time
import random
import re
from bs4 import BeautifulSoup
import pymorphy2
import pandas as pd
from sqlalchemy import create_engine, text

morph = pymorphy2.MorphAnalyzer()

## Данные

In [ ]:
# !pip install pymorphy2

### План
1. Составляем список ссылок на все рецепты с сайта
2. Проходимся по каждой ссылке и добываем оттуда ингридиенты и их количество.
3. Склываем в таблицу (mySQL) название id-блюда-ссылка-массив с ингридиентами 

### Составляем список ссылок на все рецепты с сайта

In [ ]:
def get_recipe_links(url):
    """
    Extracts recipe links from a given webpage.

    Args:
    - url (str): The URL of the webpage.

    Returns:
    - list: A list of recipe links found on the webpage.
    """
    # Ensure that 'url' is used instead of an undeclared variable 'page'
    response = requests.get(url)

    # Use try-except block to handle potential exceptions during parsing
    try:
        soup = BeautifulSoup(response.text, 'html.parser')
    except Exception as e:
        print(f"Error parsing HTML: {e}")
        return []

    links = []
    for link in soup.find_all('a'):
        recipe_link = link.get('href')

        # Ensure that 'https' is properly checked and 'recipe' is in the URL
        if recipe_link and 'https' in recipe_link and 'recipe' in recipe_link:
            links.append(recipe_link)

    return links

In [ ]:
%%time

# первая страница с ссылками на рецепты отличается от последующих
url = 'https://www.say7.info/cook/'
recipe_links = get_recipe_links(url)

# далее принцип формирования ссылок понятен: по 20 на каждой странице
for n in range(20, 1400, 20):
    url = f'https://www.say7.info/cook/linkz_start-{n}.html'
    recipe_links += get_recipe_links(url)

# записываем полученный список ссылок на рецепты в файл
with open('../data/recipes.txt', 'w') as f:
    for url in recipe_links:
        f.write(url+'\n')

# В первый раз всё прошло за 36 c

In [ ]:
# читаем файл со списком ссылок
with open('../data/recipes.txt') as f:
    recipe_urls = f.readlines()

# убираем символы перевода каретки из ссылок
recipe_urls = list(map(lambda x: x.strip(), recipe_links))

### Список ингридиентов для каждого рецепта

In [ ]:
def get_recipe_name_n_ingredients(recipe_url, headers=None):
    ingredients = []

    # Set headers to mimic a browser
    if headers is None:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

    try:
        response = requests.get(recipe_url, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {recipe_url}: {e}")
        return {'recipe_name': None, 'ingredients': []}

    soup = BeautifulSoup(response.text, 'html.parser')
    recipe_name = soup.title.text  # название рецепта

    # Extract ingredients logic here
    for item in soup.find_all('li', class_='p-ingredient'):
        ingredients.append(item.text)

    return {'recipe_url': recipe_url, 'recipe_name': recipe_name, 'ingredients': ingredients}


In [ ]:
%%time
i = 0
recipe_names = []
ingredients = []

for recipe_url in recipe_urls:
    recipe_attrs = get_recipe_name_n_ingredients(recipe_url)
    recipe_names.append(recipe_attrs['recipe_name'])
    ingredients.append(recipe_attrs['ingredients'])
    print(i, recipe_url, recipe_attrs['recipe_name'])
    
    # делаем паузу, чтоб не сильно атаковать сервер
    random_pause = random.uniform(0.5, 10.0)
    time.sleep(random_pause)
    i += 1

```
ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None))
```

Here is what I think happens: I'm trying to scrap data. The webserver does not like that and stops talking to my PC. The people owning the webserver have put measures into place that prevent random people from scraping all their stuff.

In [ ]:
df = pd.DataFrame(
    {
        'url': recipe_urls,
        'recipe_name': recipe_names,
        'ingredients': ingredients
    }
)

In [ ]:
df.to_excel('../data/base.xlsx')

In [ ]:
# df.to_csv('../data/base.csv')

In [ ]:
# собираем то, что не получилось в первый раз
recipe_urls = []
recipe_names = []
ingredients = []
i = 0

for recipe_url in df[df['recipe_name'].isna()]['url']:
    recipe_attrs = get_recipe_name_n_ingredients(recipe_url)
    recipe_urls.append(recipe_attrs['recipe_url'])
    recipe_names.append(recipe_attrs['recipe_name'])
    ingredients.append(recipe_attrs['ingredients'])
    print(i, recipe_url, recipe_attrs['recipe_name'])
    
    # делаем паузу
    random_pause = random.uniform(0.5, 5)
    time.sleep(random_pause)
    i += 1

In [ ]:
df_to_add = pd.DataFrame(
    {
        'url': recipe_urls,
        'recipe_name': recipe_names,
        'ingredients': ingredients
    }
)

In [ ]:
df.dropna(subset=['recipe_name'], inplace=True)
df = df.append(df_to_add)

In [ ]:
df.info()

In [ ]:
df.to_csv('../data/base.csv', index=None)
df.to_excel('../data/base.xlsx', index=None)

### Преобазование списка ингридиентов (начальная форма) и создание правил для поиска и замены

In [42]:
df = pd.read_excel('../data/database.xlsx')

In [43]:
df.head()

,url,recipe_name,ingredients
0,https://www.say7.info/cook/recipe/1590-Tort-Tr...,Торт «Три шоколада»,"['3 яйца (небольших)', '100\xa0г сахара', '70\..."
1,https://www.say7.info/cook/recipe/56-Salat-Mim...,Салат «Мимоза»,['200\xa0г консервированной в\xa0масле рыбы (л...
2,https://www.say7.info/cook/recipe/935-Pelmeni....,Пельмени,"['250\xa0мл молока', '1 яйцо', '1\xa0ст.л. рас..."
3,https://www.say7.info/cook/recipe/60-YAblochny...,Яблочный пирог,"['1\xa0кг яблок (желательно, кислых или\xa0кис..."
4,https://www.say7.info/cook/recipe/1521-Brusket...,Брускетта с тунцом и помидорами,"['150\xa0г багета', '150\xa0г тунца (я\xa0испо..."


#### Правила замены (упрощения)

In [161]:
def make_initial_form(ingredient):
    """
    ingredient (str): name of the ingredient
    """
    ingredient_initial = [morph.parse(word)[0].normal_form for word in ingredient.split()]
    return ' '.join(ingredient_initial)


def extract_ingredient_names(string):
    # Use regular expression to find text within single quotes
    ingredient_names = re.findall(r"'(.*?)'", string)

    # Remove digits, specific strings '\\xa0г' and '\\xa0', and the "–" symbol
    cleaned_ingredient_names = [
        re.sub(r'\d+|\\xa0г|\\xa0|–|\\xaмл|\\xa0ч.л.|\([^)]*\)|%|\\xa0ст.л.|\\xa0кг',
               '', ingredient).strip() for ingredient in ingredient_names]
    
    cleaned_ingredient_names = [
        re.sub(r'ч.л.|ст.л.|¼|½|\.',
               '', ingredient).strip() for ingredient in cleaned_ingredient_names]
    
    # Making initial form of words
    ingredients_initial = [
        make_initial_form(ingredient) for ingredient in cleaned_ingredient_names]
    
    return ingredients_initial


In [162]:
# test 1
string = "['200\\xa0г консервированной в\\xa0масле рыбы (лучше всего сайры)', '300\\xa0г картофеля', '200\\xa0г моркови', '150\\xa0г лука', '4 яйца', 'соль', 'майонез']"
extract_ingredient_names(string)

['консервированный вмасло рыба',
 'картофель',
 'морковь',
 'лука',
 'яйцо',
 'соль',
 'майонез']

In [163]:
# test 2
string = "['500\\xa0г филе семги или\\xa0форели', '50\\xa0г (2.5\\xa0ст.л.) соли (лучше крупной)', '20\\xa0г (1\\xa0ст.л.) сахара', '½\\xa0ч.л. перца (по желанию)']"
extract_ingredient_names(string)

['филе сёмга илифорель', 'соль', 'сахар', 'перец']

In [164]:
# test 3
string = "['2 яйца', '100\xa0г сахара', '1\xa0ч.л. разрыхлителя', '100–130\xa0г муки', '600\xa0г творожного сыра (Филадельфия, Хохланд, Альметте и\xa0т.д.)', '200\xa0мл сливок 33%', '3 яйца', '150\xa0г сахара', '2\xa0ч.л. ванильного сахара', '500\xa0г яблок', '1\xa0ч.л. корицы (по желанию)', '3\xa0ст.л. коричневого сахара (или белого)']"
extract_ingredient_names(string)

['яйцо',
 'г сахар',
 'разрыхлитель',
 'г мука',
 'г творожный сыр',
 'мл сливка',
 'яйцо',
 'г сахар',
 'ванильный сахар',
 'г яблоко',
 'корица',
 'коричневый сахар']

In [175]:
# string = df.iloc[122, 2]
# print(string)
# print(extract_ingredient_names(string))

In [176]:
%%time
df['ingredients_clean'] = df['ingredients'].apply(extract_ingredient_names)

Wall time: 8.49 s


In [177]:
df.head()

,url,recipe_name,ingredients,ingredients_clean
0,https://www.say7.info/cook/recipe/1590-Tort-Tr...,Торт «Три шоколада»,"['3 яйца (небольших)', '100\xa0г сахара', '70\...","[яйцо, сахар, мл молоко, сливочный масло, шоко..."
1,https://www.say7.info/cook/recipe/56-Salat-Mim...,Салат «Мимоза»,['200\xa0г консервированной в\xa0масле рыбы (л...,"[консервированный вмасло рыба, картофель, морк..."
2,https://www.say7.info/cook/recipe/935-Pelmeni....,Пельмени,"['250\xa0мл молока', '1 яйцо', '1\xa0ст.л. рас...","[мл молоко, яйцо, растительный масло, соль, му..."
3,https://www.say7.info/cook/recipe/60-YAblochny...,Яблочный пирог,"['1\xa0кг яблок (желательно, кислых или\xa0кис...","[кг яблоко, сахар, сливочный масло, яйцо, разр..."
4,https://www.say7.info/cook/recipe/1521-Brusket...,Брускетта с тунцом и помидорами,"['150\xa0г багета', '150\xa0г тунца (я\xa0испо...","[багет, тунец, помидор, зелень повкус, соль, р..."


In [178]:
all_ingredients = []

for ingredient_list in df['ingredients_clean']:
    for ingredient in ingredient_list:
        all_ingredients.append(ingredient)
        
all_ingredients = pd.Series(all_ingredients)

In [179]:
all_ingredients.describe()

count     11499
unique     1064
top        соль
freq        979
dtype: object

In [180]:
# all_ingredients.value_counts().to_excel('../data/unified_names.xlsx', index=None)
all_ingredients.value_counts().to_excel('../data/name_mapping.xlsx', index=True)

То, что наговаривает пользователи (как называет свои продукты), тоже нужно будет записвыать и анализировать, чтобы понять, какие им замены подбирать. 

In [181]:
df_unified_names = pd.read_excel('../data/unified_names.xlsx')
df_unified_names.head()

,possible_name,qty,unified_name,is_necessary
0,лук,451.0,лук,True
1,соль,867.0,соль,False
2,перец,691.0,перец,True
3,сахар,532.0,сахар,False
4,яйцо,523.0,яйцо,True


In [183]:
df_unified_names

,possible_name,qty,unified_name,is_necessary
0,лук,451.0,лук,True
1,соль,867.0,соль,False
2,перец,691.0,перец,True
3,сахар,532.0,сахар,False
4,яйцо,523.0,яйцо,True
...,...,...,...,...
3104,сладкое молотый перец,1.0,NaN,NaN
3105,крупный ракушка лумаконь,1.0,NaN,NaN
3106,филе сырой сёмга илифорель,1.0,NaN,NaN
3107,мл сметана,1.0,NaN,NaN


In [184]:
# df_unified_names.drop_duplicates(subset='possible_name').to_excel('../data/name_mapping.xlsx', index=False)

In [37]:
# Dictionary defining the mappings
name_mappings = dict(zip(df_unified_names['possible_name'], df_unified_names['unified_name']))


# Replace with unified names
def unify_ingredient_names(ingredient_list, name_mappings):
    mapped_list = [name_mappings.get(ingredient, ingredient) for ingredient in ingredient_list]
    return mapped_list

df['ingredients_unified'] = df['ingredients_clean'].apply(
    unify_ingredient_names, **{'name_mappings': name_mappings}
)

df.drop(columns=['ingredients', 'ingredients_clean'], inplace=True)

In [41]:
df.head()

,url,recipe_name,ingredients_unified
0,https://www.say7.info/cook/recipe/1590-Tort-Tr...,Торт «Три шоколада»,"[яйцо, сахар, молоко, сливочный масло, шоколад..."
1,https://www.say7.info/cook/recipe/56-Salat-Mim...,Салат «Мимоза»,[консервированный вмасло рыба (хороший весь са...
2,https://www.say7.info/cook/recipe/935-Pelmeni....,Пельмени,"[молоко, яйцо, растительное масло, соль, мука,..."
3,https://www.say7.info/cook/recipe/60-YAblochny...,Яблочный пирог,"[кг яблоко (желательно, кислый иликисло-сладки..."
4,https://www.say7.info/cook/recipe/1521-Brusket...,Брускетта с тунцом и помидорами,"[багет, тунец (яиспользовать всобственный соку..."


### Создание базы данных SQLite

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html

In [7]:
engine = create_engine('sqlite://', echo=False)
df.to_sql(name='recipes.sqlite', con=engine)

TypeError: __init__() got multiple values for argument 'schema'

In [ ]:
# TypeError: __init__() got multiple values for argument 'schema'
#  this is an indication of a mismatch between pandas and SQLAlchemy version.
# It is working on these fixed versions.
# pandas==1.1.5 SQLAlchemy==1.4.45

In [ ]:
# with engine.connect() as conn:
#     conn.execute(text("SELECT * FROM users")).fetchall()